<center><font size='20' color='red'>Taller de Visualizaciones</font></center>

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium, fiona, branca

In [2]:
df = pd.read_csv(
    '../Data/ec_properties.csv',
    parse_dates=['start_date', 'end_date'],
)

1. <font color='green'>Filtra el dataset para quedarse con aquellas casas que tienen latitud, longitud, y precio.</font>

In [3]:
df = df[df['lon'].notnull() & df['lat'].notnull() & df['price'].notnull() & (df['l3'] == 'Guayaquil') & (df['price'] > 0) & (df['property_type'] == 'Casa') & (df['start_date'].dt.year == 2020) & (df['operation_type'] == 'Venta')]

2. <font color='green'>Crea un dataframe grográfico con la información de las casas.</font>

In [4]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lon'], df['lat']))

/Users/alfreed0/opt/anaconda3/envs/geo_solver/lib/python3.9/site-packages/geopandas/array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)


In [5]:
zonas = gpd.read_file('../Data/zonas_inec.geojson')

In [6]:
zonas = zonas[zonas['DPA_NOMBRE'] == 'GUAYAQUIL']

In [7]:
gdf.crs = zonas.crs

3. <font color='green'>Realiza una intersección espacial entre el dataframe geográfico de las casas y el dataframe geográfico de los sectores censales.</font>

In [8]:
dfr = gpd.sjoin(gdf, zonas, how='inner', op='intersects')

4. <font color='green'>Realiza una agrupación por sectores censales y calcula los precios promedios de las casas.</font>

In [9]:
choro = dfr.groupby('DPA_ZONA')['price'].mean().reset_index()

In [10]:
choro = choro.merge(zonas , on='DPA_ZONA')

In [11]:
choro = gpd.GeoDataFrame(choro, geometry='geometry')

In [12]:
choro.to_file('../Output/ShapeFiles/choro.geojson', driver='GeoJSON')

/Users/alfreed0/opt/anaconda3/envs/geo_solver/lib/python3.9/site-packages/geopandas/io/file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [13]:
centroid=dfr.geometry.centroid
mapbox_api_key = r"pk.eyJ1Ijoic3RhbWVuIiwiYSI6IlpkZEtuS1EifQ.jiH_c9ShtBwtqH9RdG40mw"; tileset_ID_str = "streets-v11"; tilesize_pixels = "256";
m=folium.Map(location=[centroid.y.mean(), centroid.x.mean()], zoom_start=12,
             tiles = f"https://api.mapbox.com/styles/v1/mapbox/{tileset_ID_str}/tiles/{tilesize_pixels}/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_api_key}",
             attr='MAPBOX')

/var/folders/bq/_bqydjv50jg7scm3v5rwy0280000gn/T/ipykernel_99885/4076983077.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid=dfr.geometry.centroid


5. <font color='green'>Realiza un mapa coroplético con los precios promedios de los sectores censales.</font>

In [14]:
field = "price"
label = "precio por sector"
colormap = branca.colormap.linear.YlOrBr_04.to_step(4).scale(choro[field].min(),choro[field].max())
colormap.caption = label
layer = folium.GeoJson('../Output/ShapeFiles/choro.geojson',
        name=label,
        style_function=lambda x: {"weight":2, 'color':'black','fillColor':colormap(x['properties'][field]), 'fillOpacity':1},
        highlight_function=lambda x: {'weight':5, 'color':'black'},
        smooth_factor=2.0,
        tooltip=folium.features.GeoJsonTooltip(fields=['DPA_ZONA',field],
                                      aliases=['DPA_ZONA',label], 
                                      labels=True, 
                                      sticky=True,
                                      localize=True
                                     ),
        show = True,
        overlay = True
        )
m.add_child(layer)
m.add_child(colormap)
m.save('../Output/ShapeFiles/choro.html')